# Level 4: Network Analysis

**Мета:** Побудова графів зв'язків між учасниками закупівель для виявлення підозрілих структур (картелів, монопольних відносин).

## Типи графів:

| Тип графа | Вузли | Ребра | Що виявляє |
|-----------|-------|-------|------------|
| **Winner-Loser** | bidder_id | Переможець → Програвший | Bid-rigging rings |
| **Buyer-Supplier** | buyer_id, supplier_id | Контракт | Монопольні відносини |
| **Bidder Co-participation** | bidder_id | Спільна участь | Потенційні картелі |
| **Full Collusion Graph** | bidder + supplier + buyer | Всі зв'язки | Комплексні схеми |

## Метрики мережі:

| Метрика | Що показує | Інтерпретація аномалії |
|---------|------------|------------------------|
| Community detection (Louvain) | Групи тісно пов'язаних учасників | Потенційні картелі |
| Betweenness centrality | Вузли-посередники | Координатори змов |
| Clustering coefficient | Щільність локальних зв'язків | Замкнуті групи |
| Degree distribution | Розподіл кількості зв'язків | Аномально активні учасники |

## Pipeline:
1. Rule-based ✓
2. Statistical ✓
3. ML (IF, HDBSCAN) ✓
4. **Network Analysis** ← current
5. Ensemble

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import defaultdict
from itertools import combinations
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

from src.data_loader import load_tenders, load_bids, load_buyers, load_suppliers, load_bidders
from src.detectors import NetworkAnalysisDetector

# ============================================================
# CONFIGURATION
# ============================================================
YEARS = [2022, 2023, 2024, 2025]
MIN_CO_BIDS = 3           # Мінімум спільних тендерів для ребра
MIN_CONTRACTS = 3         # Мінімум контрактів для buyer-supplier ребра
MIN_COMMUNITY_SIZE = 3    # Мінімальний розмір community
# ============================================================

# Create output directories
Path('../results/figures/network').mkdir(parents=True, exist_ok=True)

# Style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

print(f"Configuration:")
print(f"  YEARS = {YEARS}")
print(f"  MIN_CO_BIDS = {MIN_CO_BIDS}")
print(f"  MIN_CONTRACTS = {MIN_CONTRACTS}")

## 1. Завантаження даних

In [ ]:
print("Loading data...")
tenders = load_tenders(years=YEARS)
bids = load_bids(years=YEARS)
buyers = load_buyers()
suppliers = load_suppliers()
bidders = load_bidders()

print(f"\nDataset:")
print(f"  Tenders: {len(tenders):,}")
print(f"  Bids: {len(bids):,}")
print(f"  Buyers: {len(buyers):,}")
print(f"  Suppliers: {len(suppliers):,}")
print(f"  Bidders: {len(bidders):,}")

In [ ]:
# Filter to competitive tenders (Open/Selective with 2+ bidders)
competitive_tenders = tenders[
    (tenders['procurement_method'].isin(['open', 'selective'])) &
    (tenders['number_of_tenderers'] >= 2)
].copy()

print(f"Competitive tenders: {len(competitive_tenders):,} ({len(competitive_tenders)/len(tenders)*100:.1f}%)")

# Filter bids to competitive tenders
bids_competitive = bids[bids['tender_id'].isin(competitive_tenders['tender_id'])].copy()
print(f"Bids in competitive tenders: {len(bids_competitive):,}")

## 2. Graph 1: Bidder Co-participation (Cartel Detection)

**Вузли:** bidder_id  
**Ребра:** Спільна участь у тендері  
**Що виявляє:** Потенційні картелі - групи постачальників, що систематично беруть участь разом

In [ ]:
# Run Network Analysis using detector class
print("Running Network Analysis Detector...")

network_detector = NetworkAnalysisDetector(
    min_co_bids=MIN_CO_BIDS,
    min_contracts=MIN_CONTRACTS,
    min_community_size=MIN_COMMUNITY_SIZE,
    # Stricter anomaly thresholds
    suspicious_min_degree=10,
    suspicious_min_clustering=0.7,
    rotation_min_ratio=0.7,
    rotation_min_interactions=5,
    monopoly_min_ratio=0.9,
    monopoly_min_contracts=20,
)

# Run detection
network_results = network_detector.fit_detect(
    tenders,
    bids_df=bids,
    bidders_df=bidders,
)

print("\nNetwork Analysis complete!")
print(network_detector.summary().to_string(index=False))

In [ ]:
# Community detection results from detector
G_cobid = network_detector.G_cobid
partition = network_detector.cobid_communities

if partition:
    # Count communities
    community_counts = pd.Series(partition).value_counts()
    n_communities = len(community_counts)

    print(f"Communities found: {n_communities}")
    print(f"Largest community: {community_counts.iloc[0]} bidders")
    print(f"Communities with 3+ bidders: {len(community_counts[community_counts >= 3])}")
else:
    print("No communities found")
    community_counts = pd.Series()

In [ ]:
# Network metrics from detector
cobid_metrics = network_detector.cobid_metrics

if cobid_metrics is not None and len(cobid_metrics) > 0:
    # Degree distribution
    degree_df = cobid_metrics[['bidder_id', 'degree']].sort_values('degree', ascending=False)
    
    print(f"\nDEGREE DISTRIBUTION (co-bidding partners):")
    print(f"  Max: {degree_df['degree'].max()}")
    print(f"  Mean: {degree_df['degree'].mean():.1f}")
    print(f"  Median: {degree_df['degree'].median():.0f}")
    
    # Top bidders by degree (most connected)
    print(f"\nTOP 10 MOST CONNECTED BIDDERS:")
    top_degree = degree_df.head(10).merge(
        suppliers[['supplier_id', 'supplier_name']].rename(columns={'supplier_id': 'bidder_id'}),
        on='bidder_id', how='left'
    )
    for _, row in top_degree.iterrows():
        name = row['supplier_name'][:50] if pd.notna(row.get('supplier_name')) else 'N/A'
        print(f"  {row['degree']:>4} partners: {name}")
else:
    print("No metrics computed")
    degree_df = pd.DataFrame()

In [ ]:
# Betweenness centrality from detector metrics
if cobid_metrics is not None and len(cobid_metrics) > 0:
    betweenness_df = cobid_metrics[['bidder_id', 'betweenness']].sort_values('betweenness', ascending=False)
    
    print(f"TOP 10 BROKERS (highest betweenness centrality):")
    print("These bidders connect different groups - potential cartel coordinators")
    
    top_betweenness = betweenness_df.head(10).merge(
        suppliers[['supplier_id', 'supplier_name']].rename(columns={'supplier_id': 'bidder_id'}),
        on='bidder_id', how='left'
    )
    for _, row in top_betweenness.iterrows():
        name = row['supplier_name'][:50] if pd.notna(row.get('supplier_name')) else 'N/A'
        print(f"  {row['betweenness']:.4f}: {name}")
else:
    print("No betweenness metrics available")
    betweenness_df = pd.DataFrame()

In [ ]:
# Clustering coefficient from detector metrics
if cobid_metrics is not None and len(cobid_metrics) > 0:
    clustering_df = cobid_metrics[['bidder_id', 'clustering']].sort_values('clustering', ascending=False)
    
    # High clustering + high degree = tightly connected group (potential cartel)
    suspicious_bidders = cobid_metrics[
        (cobid_metrics['degree'] >= 5) &  # At least 5 partners
        (cobid_metrics['clustering'] >= 0.5)  # Partners also connected to each other
    ].sort_values(['clustering', 'degree'], ascending=[False, False])
    
    print(f"\nSUSPICIOUS BIDDERS (high degree + high clustering):")
    print(f"Found: {len(suspicious_bidders)} bidders in tightly connected groups")
    
    if len(suspicious_bidders) > 0:
        suspicious_with_names = suspicious_bidders.head(10).merge(
            suppliers[['supplier_id', 'supplier_name']].rename(columns={'supplier_id': 'bidder_id'}),
            on='bidder_id', how='left'
        )
        for _, row in suspicious_with_names.iterrows():
            name = row['supplier_name'][:40] if pd.notna(row.get('supplier_name')) else 'N/A'
            print(f"  Degree: {row['degree']:>3}, Clustering: {row['clustering']:.2f} - {name}")
else:
    print("No clustering metrics available")
    suspicious_bidders = pd.DataFrame()

In [ ]:
# Visualize co-bidding network analysis
print("Visualizing co-bidding network...")

fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# Plot 1: Degree distribution
ax = axes[0, 0]
if len(degree_df) > 0:
    ax.hist(degree_df['degree'], bins=50, color='steelblue', edgecolor='white')
    ax.set_xlabel('Degree (number of co-bidding partners)')
    ax.set_ylabel('Count')
    ax.set_title('Degree Distribution')
    ax.set_yscale('log')
else:
    ax.text(0.5, 0.5, 'No data', ha='center', va='center')

# Plot 2: Community size distribution
ax = axes[0, 1]
if len(community_counts) > 0:
    ax.bar(range(min(20, len(community_counts))), community_counts.head(20).values, color='coral')
    ax.set_xlabel('Community Rank')
    ax.set_ylabel('Size (number of bidders)')
    ax.set_title('Top 20 Communities by Size')
else:
    ax.text(0.5, 0.5, 'No communities', ha='center', va='center')

# Plot 3: Clustering vs Degree
ax = axes[1, 0]
if len(suspicious_bidders) > 0:
    sample = suspicious_bidders if len(suspicious_bidders) < 1000 else suspicious_bidders.sample(1000)
    ax.scatter(sample['degree'], sample['clustering'], alpha=0.5, s=20)
    ax.axhline(0.5, color='red', linestyle='--', alpha=0.5)
    ax.axvline(5, color='red', linestyle='--', alpha=0.5)
ax.set_xlabel('Degree')
ax.set_ylabel('Clustering Coefficient')
ax.set_title('Clustering vs Degree (suspicious zone: high both)')

# Plot 4: Largest community subgraph
ax = axes[1, 1]
if partition and len(community_counts) > 0:
    top_community_ids = community_counts[community_counts >= 5].head(10).index.tolist()
    if len(top_community_ids) > 0:
        largest_comm_nodes = [n for n, c in partition.items() if c == top_community_ids[0]]
        subgraph = G_cobid.subgraph(largest_comm_nodes[:50])  # Limit for visualization
        
        pos = nx.spring_layout(subgraph, k=2, iterations=50, seed=42)
        nx.draw_networkx_nodes(subgraph, pos, node_size=50, node_color='coral', alpha=0.8, ax=ax)
        nx.draw_networkx_edges(subgraph, pos, alpha=0.3, ax=ax)
        ax.set_title(f'Largest Community ({len(largest_comm_nodes)} bidders, showing 50)')
ax.axis('off')

plt.tight_layout()
plt.savefig('../results/figures/network/cobid_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Graph 2: Winner-Loser (Bid Rigging Detection)

**Вузли:** bidder_id  
**Ребра:** Направлені: Переможець → Програвший  
**Що виявляє:** Bid-rigging rings - якщо A завжди виграє у B, а B завжди виграє у C, а C у A

In [ ]:
# Winner-Loser graph from detector
G_winlose = network_detector.G_winlose

if G_winlose is not None:
    print(f"Winner-Loser Graph:")
    print(f"  Nodes: {G_winlose.number_of_nodes():,}")
    print(f"  Edges (with {MIN_CO_BIDS}+ wins): {G_winlose.number_of_edges():,}")
else:
    print("Winner-Loser graph not built")

In [ ]:
# Bid rotation pairs from detector
rotation_df = network_detector.get_rotation_pairs(min_ratio=0.3)

print("BID ROTATION ANALYSIS:")
print("="*80)

if len(rotation_df) > 0:
    suspicious_rotation = rotation_df[rotation_df['rotation_ratio'] >= 0.5]
    
    print(f"\nBID ROTATION PAIRS (both win against each other):")
    print(f"Found: {len(suspicious_rotation)} suspicious pairs (rotation >= 50%)")
    
    for _, row in suspicious_rotation.head(10).iterrows():
        b1_name = suppliers[suppliers['supplier_id'] == row['bidder_1']]['supplier_name'].values
        b2_name = suppliers[suppliers['supplier_id'] == row['bidder_2']]['supplier_name'].values
        name1 = b1_name[0][:30] if len(b1_name) > 0 else 'N/A'
        name2 = b2_name[0][:30] if len(b2_name) > 0 else 'N/A'
        
        print(f"\n  Rotation: {row['rotation_ratio']:.0%}")
        print(f"    {name1} wins {row['wins_1_over_2']}x")
        print(f"    {name2} wins {row['wins_2_over_1']}x")
else:
    print("No reciprocal win-lose pairs found")

In [ ]:
# Find cycles (bid rigging rings)
print("Searching for bid rigging rings (cycles)...")

if G_winlose is not None and G_winlose.number_of_nodes() > 0:
    try:
        # Limit search to prevent memory issues
        cycles_3 = []
        for cycle in nx.simple_cycles(G_winlose, length_bound=3):
            if len(cycle) == 3:
                cycles_3.append(cycle)
            if len(cycles_3) >= 100:  # Limit
                break
        
        print(f"Found {len(cycles_3)} triangular rings (A→B→C→A)")
        
        if len(cycles_3) > 0:
            print("\nTOP 5 BID RIGGING RINGS:")
            for i, cycle in enumerate(cycles_3[:5]):
                print(f"\n  Ring {i+1}:")
                for bidder_id in cycle:
                    name = suppliers[suppliers['supplier_id'] == bidder_id]['supplier_name'].values
                    name = name[0][:40] if len(name) > 0 else 'N/A'
                    print(f"    - {name}")
    except Exception as e:
        print(f"Cycle detection skipped: {e}")
else:
    print("No winner-loser graph available")

## 4. Graph 3: Buyer-Supplier (Monopoly Detection)

**Вузли:** buyer_id, supplier_id  
**Ребра:** Контракт  
**Що виявляє:** Монопольні відносини, підозрілі зв'язки

In [ ]:
# Buyer-Supplier graph from detector
G_bs = network_detector.G_buyer_supplier

if G_bs is not None:
    print(f"Buyer-Supplier Graph:")
    print(f"  Nodes: {G_bs.number_of_nodes():,}")
    print(f"  Edges: {G_bs.number_of_edges():,}")
else:
    print("Buyer-Supplier graph not built")

In [ ]:
# Monopolistic relationships from detector
monopolistic = network_detector.get_monopolistic_relationships()

print("Analyzing monopolistic relationships...")
print(f"\nMONOPOLISTIC RELATIONSHIPS (one supplier gets 80%+ contracts):")
print(f"Found: {len(monopolistic)} buyer-supplier pairs")

if len(monopolistic) > 0:
    monopolistic_with_names = monopolistic.head(10).merge(
        buyers[['buyer_id', 'buyer_name']], on='buyer_id', how='left'
    ).merge(
        suppliers[['supplier_id', 'supplier_name']], on='supplier_id', how='left'
    )
    
    for _, row in monopolistic_with_names.iterrows():
        buyer_name = row['buyer_name'][:30] if pd.notna(row.get('buyer_name')) else 'N/A'
        supplier_name = row['supplier_name'][:30] if pd.notna(row.get('supplier_name')) else 'N/A'
        print(f"\n  Dominance: {row['dominance_ratio']:.0%}, Value: {row['total_value']/1e6:.1f}M UAH")
        print(f"    Buyer: {buyer_name}")
        print(f"    Supplier: {supplier_name}")

In [ ]:
# Community detection in buyer-supplier network (separate analysis)
print("Detecting communities in buyer-supplier network...")

if G_bs is not None and G_bs.number_of_nodes() > 0:
    try:
        from community import community_louvain
        bs_partition = community_louvain.best_partition(G_bs, weight='weight')
    except ImportError:
        from networkx.algorithms.community import greedy_modularity_communities
        bs_communities = list(greedy_modularity_communities(G_bs, weight='weight'))
        bs_partition = {}
        for i, comm in enumerate(bs_communities):
            for node in comm:
                bs_partition[node] = i

    bs_community_counts = pd.Series(bs_partition).value_counts()

    print(f"Communities found: {len(bs_community_counts)}")
    print(f"Largest community: {bs_community_counts.iloc[0]} nodes")

    # Analyze mixed communities (buyers and suppliers together)
    print("\nAnalyzing community composition...")
    for comm_id in bs_community_counts.head(5).index:
        comm_nodes = [n for n, c in bs_partition.items() if c == comm_id]
        buyers_in_comm = [n for n in comm_nodes if n.startswith('B_')]
        suppliers_in_comm = [n for n in comm_nodes if n.startswith('S_')]
        print(f"  Community {comm_id}: {len(buyers_in_comm)} buyers, {len(suppliers_in_comm)} suppliers")
else:
    print("No buyer-supplier graph available")

## 5. Summary & Export

In [ ]:
print("="*60)
print("NETWORK ANALYSIS SUMMARY")
print("="*60)

print(f"\nDATASET:")
print(f"  Tenders: {len(tenders):,}")
print(f"  Competitive tenders: {len(competitive_tenders):,}")
print(f"  Bids: {len(bids_competitive):,}")

# Summary from detector
summary = network_detector.summary()
print(f"\nGRAPH METRICS:")
for _, row in summary.iterrows():
    print(f"  {row['metric']}: {int(row['value']):,}")

print(f"\nKEY FINDINGS:")
if len(suspicious_bidders) > 0:
    print(f"  Suspicious bidders (high degree + clustering): {len(suspicious_bidders)}")
if len(rotation_df) > 0:
    print(f"  Bid rotation pairs: {len(rotation_df[rotation_df['rotation_ratio'] >= 0.5])}")
if len(monopolistic) > 0:
    print(f"  Monopolistic relationships: {len(monopolistic)}")

print("\n" + "="*60)

In [ ]:
# Export results
print("Saving results...")

# 1. Network results (tender-level)
network_results.to_csv('../results/network_results.csv', index=False)
print(f"Saved: network_results.csv ({len(network_results)} rows)")

# 2. Suspicious bidders (high connectivity)
if len(suspicious_bidders) > 0:
    suspicious_export = suspicious_bidders.merge(
        suppliers[['supplier_id', 'supplier_name']].rename(columns={'supplier_id': 'bidder_id'}),
        on='bidder_id', how='left'
    )
    suspicious_export['community'] = suspicious_export['bidder_id'].map(partition)
    suspicious_export.to_csv('../results/network_suspicious_bidders.csv', index=False)
    print(f"Saved: network_suspicious_bidders.csv ({len(suspicious_export)} rows)")

# 3. Bid rotation pairs
if len(rotation_df) > 0:
    rotation_export = rotation_df[rotation_df['rotation_ratio'] >= 0.3].copy()
    rotation_export = rotation_export.merge(
        suppliers[['supplier_id', 'supplier_name']].rename(
            columns={'supplier_id': 'bidder_1', 'supplier_name': 'bidder_1_name'}
        ), on='bidder_1', how='left'
    ).merge(
        suppliers[['supplier_id', 'supplier_name']].rename(
            columns={'supplier_id': 'bidder_2', 'supplier_name': 'bidder_2_name'}
        ), on='bidder_2', how='left'
    )
    rotation_export.to_csv('../results/network_bid_rotation.csv', index=False)
    print(f"Saved: network_bid_rotation.csv ({len(rotation_export)} rows)")

# 4. Monopolistic relationships
if len(monopolistic) > 0:
    monopolistic_export = monopolistic.merge(
        buyers[['buyer_id', 'buyer_name']], on='buyer_id', how='left'
    ).merge(
        suppliers[['supplier_id', 'supplier_name']], on='supplier_id', how='left'
    )
    monopolistic_export.to_csv('../results/network_monopolistic.csv', index=False)
    print(f"Saved: network_monopolistic.csv ({len(monopolistic_export)} rows)")

# 5. Community assignments
if cobid_metrics is not None and len(cobid_metrics) > 0:
    community_export = cobid_metrics.copy()
    community_export['community'] = community_export['bidder_id'].map(partition)
    community_export = community_export.merge(
        suppliers[['supplier_id', 'supplier_name']].rename(columns={'supplier_id': 'bidder_id'}),
        on='bidder_id', how='left'
    )
    community_export.to_csv('../results/network_communities.csv', index=False)
    print(f"Saved: network_communities.csv ({len(community_export)} rows)")

print("\nDone!")

## Висновки

### Що виявлено:

1. **Co-bidding Network**
   - Communities = потенційні картелі
   - High betweenness = координатори змов
   - High clustering + degree = тісно пов'язані групи

2. **Winner-Loser Network**
   - Reciprocal edges (A beats B AND B beats A) = bid rotation
   - Cycles = bid rigging rings

3. **Buyer-Supplier Network**
   - Monopolistic relationships = один supplier домінує
   - Communities = групи пов'язаних buyers та suppliers

### Для Ensemble:
- Network features можна додати до тендерів (is_in_cartel_community, supplier_monopoly_score, etc.)
- Або використовувати network anomalies як окремий метод у cross-method validation